### расчет опасных деревьев  
есть массив точек предположительно вершин опасных деревьев,  
есть также большой массив земли. 
- нужно разделить массивы попролетно (добавить семантику в таблицы?)  
- затем для каждого дерева найти координату на земле
- затем определить для каждого дерева расстояние до провода
(координаты опор и ТК проводов берутся из отдельных таблиц)

In [81]:
from pathlib import Path
import pandas as pd
import numpy as np
from shapely.geometry import Point, Polygon, LineString
import csv

from math import sqrt, acos, atan2, degrees
import matplotlib.pyplot as plt
from shapely.ops import split
from descartes import PolygonPatch

In [76]:
wide = 60 # ширина коридора поиска точек
max_span = 600 # длина максимального пролета не более
dt_buf = 4   # буффер поиска земли под деревом

In [23]:
# пути 
workdir = Path(r'D:\work\2022_m\110_man-nps1\test\ohzone')
grddir = Path(r'D:\work\2022_m\110_man-nps1\test\ohzone\grd')

In [8]:
cgt_tab = pd.read_csv(workdir / 'area.txt', delimiter='\t', header=0, encoding = 'cp1251')

In [9]:
cgt_tab

,id,x,y,z,ltk_x,ltk_y,rtk_x,rtk_y,area_c,area
0,ПС_Мангазея,557439.52,7388308.80,19.58,557442.740,7388315.150,557437.300,7388302.250,"[(557439.52, 7388308.8), (557452.187522399, 73...",1.605071e+03
1,1,557468.30,7388296.38,18.98,557468.825,7388301.170,557467.855,7388291.620,"[(557468.3, 7388296.38), (557464.7857984653, 7...",8.521589e+03
2,2,557639.36,7388332.05,19.05,557638.410,7388336.140,557640.310,7388327.960,"[(557639.36, 7388332.05), (557634.2847406843, ...",1.253662e+04
3,3,557890.35,7388384.45,18.88,557890.360,7388389.250,557890.340,7388379.650,"[(557890.35, 7388384.45), (557894.5454659202, ...",1.151289e+04
4,4,558117.80,7388335.67,17.08,558118.805,7388340.360,558116.795,7388330.980,"[(558117.8, 7388335.67), (558123.090503863, 73...",5.399490e+03
...,...,...,...,...,...,...,...,...,...,...
195,195,572291.98,7359966.88,14.06,572286.650,7359971.655,572296.590,7359961.975,"[(572291.98, 7359966.88), (572267.0996449698, ...",1.820016e+03
196,ПС_НПС-1,572300.04,7360000.50,16.41,572293.120,7360001.650,572306.630,7359998.400,"[(572300.04, 7360000.5), (572275.2324662302, 7...",1.340810e+06
197,4c,558117.80,7388335.67,17.08,558119.880,7388340.130,558117.870,7388330.750,"[(558117.8, 7388335.67), (558135.9962206875, 7...",3.569346e+03
198,4a,558162.26,7388275.24,18.53,558164.285,7388279.705,558160.235,7388270.775,"[(558162.26, 7388275.24), (558162.5315878628, ...",8.349358e+03


In [112]:
# определим азимут и расстояние

def azimuth(a, b):
    # by two points we get azimuth
    dx = a[0] - b[0]
    dy = a[1] - b[1]

    a2 = degrees(atan2(dx, dy))
    

    return a2 + 180


In [10]:
def dista(a, b):
    # dist a to b
    
    dx = a[0] - b[0]
    dy = a[1] - b[1]
    dist = round(sqrt(dx*dx + dy*dy), 2)    

    return dist

In [12]:
for n in range(len(cgt_tab)-1):
    idx = cgt_tab.iloc[n].name
    struct = (cgt_tab.loc[idx, 'x'], cgt_tab.loc[idx, 'y'])
    next_struct = (cgt_tab.loc[idx+1, 'x'], cgt_tab.loc[idx+1, 'y'])
    
    print(cgt_tab.loc[idx, 'id'], dista(struct, next_struct))


ПС_Мангазея 31.35
1 174.74
2 256.4
3 232.62
4 110.09
5 110.01
6 209.76
7 219.89
8 185.07
9 247.8
10 253.79
11 252.91
12 252.95
13 251.9
14 250.81
15 249.89
16 248.96
17 247.95
18 246.84
19 245.92
20 244.9
21 243.96
22 242.96
23 241.86
24 208.56
25 201.23
26 201.81
27 197.01
28 256.9
29 255.84
30 254.94
31 253.92
32 252.87
33 251.91
34 250.94
35 250.85
36 249.94
37 248.96
38 202.41
39 198.38
40 238.8
41 237.93
42 236.88
43 235.95
44 234.8
45 233.99
46 232.89
47 231.9
48 230.95
49 229.85
50 211.89
51 224.95
52 254.95
53 254.94
54 244.85
55 262.9
56 238.87
57 234.88
58 233.9
59 232.95
60 231.94
61 231.91
62 201.23
63 209.5
64 233.88
65 232.98
66 231.92
67 230.93
68 230.89
69 230.94
70 229.97
71 228.86
72 227.93
73 226.86
74 225.93
75 224.9
76 194.94
77 194.96
78 194.42
79 189.46
80 259.87
81 250.9
82 264.95
83 187.98
84 240.73
85 299.94
86 219.92
87 299.79
88 269.96
89 241.43
90 250.27
91 142.91
92 234.81
93 288.0
94 246.79
95 277.53
96 342.29
97 179.96
98 179.96
99 183.12
100 212.03
101 

грузим точки и для каждой определяем в каком она пролете

In [15]:
dt_tab = pd.read_csv(workdir / 'dt.xyz', delimiter=' ', header=None, names = ['x', 'y', 'z'], encoding = 'cp1251')

In [39]:
for dt in range(len(dt_tab)):
    dt_idx = dt_tab.iloc[dt].name
    dt_point = (dt_tab.iloc[dt].x, dt_tab.iloc[dt].y)
    
    for n in range(len(cgt_tab)-1):
        idx = cgt_tab.iloc[n].name
        struct = (cgt_tab.loc[idx, 'x'], cgt_tab.loc[idx, 'y'])
        next_struct = (cgt_tab.loc[idx+1, 'x'], cgt_tab.loc[idx+1, 'y'])
        span = LineString([struct, next_struct])
        if Point(dt_point).distance(span) < wide:
            span_id = cgt_tab.loc[idx, 'id']
            span_id_end = cgt_tab.loc[idx+1, 'id']
            break
    dt_tab.loc[dt_idx, 'span_id'] = span_id
    dt_tab.loc[dt_idx, 'span_id_end'] = span_id_end
        

In [40]:
dt_tab[55:59]

,x,y,z,span_id,next_span_id,span_id_end
55,575266.58,7374581.59,29.36,129,130,130
56,575268.14,7374580.85,29.69,129,130,130
57,575267.14,7374583.49,26.42,129,130,130
58,575266.54,7374584.22,24.90,129,130,130


далее опять проходим и ищем землю под каждой точкой.  
землю записываем в директорию grd

In [72]:
grd_blocks = []   # list of available ground blocks

In [70]:
def points_load(area, array_path, out_path):
    new_array = []
    with open(array_path) as grd_file:
        for line in grd_file:
            x_point = Point(float(line.strip('\n').split(' ')[0]), float(line.strip('\n').split(' ')[1]))
            if dt_area.contains(x_point):
                new_array.append(line)

    with out_path.open("w", encoding ="utf-8") as gf:
        for line in new_array:
            gf.write(line)


In [71]:
#for dt in range(len(dt_tab)):
for dt in range(1):
    dt_idx = dt_tab.iloc[dt].name
    dt_point = (dt_tab.iloc[dt].x, dt_tab.iloc[dt].y)
    dt_span = dt_tab.iloc[dt].span_id
    dt_span_end = dt_tab.iloc[dt].span_id_end
    span_start = (float(cgt_tab.loc[cgt_tab['id'] == dt_span].x), float(cgt_tab.loc[cgt_tab['id'] == dt_span].y))
    span_end = (float(cgt_tab.loc[cgt_tab['id'] == dt_span_end].x), float(cgt_tab.loc[cgt_tab['id'] == dt_span_end].y))
    span_line = LineString([span_start, span_end])
    
    if dt_span not in grd_blocks:
        dt_area = span_line.buffer(wide, cap_style=2)
        
        # загружаем в блок точки из файла земли и добавляем в лист
        points_load(dt_area, workdir/'grd_cut.xyz', grddir/dt_span)
        grd_blocks.append(dt_span)

    # дальше для той точки строим буффер и отбираем в него точки земли
    dt_z_list = []   # сюда соберем высоты земли
    dt_point_buf = Point(dt_point).buffer(dt_buf)
    with open(grddir/dt_span) as grd_file:
        for line in grd_file:
            x_point = Point(float(line.strip('\n').split(' ')[0]), float(line.strip('\n').split(' ')[1]))
            if dt_point_buf.contains(x_point):
                dt_z_list.append(float(line.strip('\n').split(' ')[2]))
                
    dt_tab.loc[dt_idx, 'dt_z'] = sum(dt_z_list) / len(dt_z_list)  # add z of tree bottom
    
    # дальше определяем слева точка или справа от линии 
    # для этого найдем азимут пролета и азимут на точку, если на точку меньше то она слева
    if azimuth(span_start, span_end) < azimuth(span_start, dt_point):
        side = r
    else:
        side = l
    # дальше расстояние до плоскости провода
    # пишем высоту, расст до оси провода, расст до оси линии, расст до опоры 
    # (потом рассчитать стейшен)

